## **Maestría en Inteligencia Artificial Aplicada**
### **Curso: Inteligencia Artificial y Aprendizaje Automático**
#### Tecnológico de Monterrey
####P rof Luis Eduardo Falcón Morales

### **Adtividad de la Semana 8**
#### **Reducción de Dimensionalidad - Descomposición en Valores Singulares (SVD) y Sistemas de Recomendación**


**Esta Actividad deberás resolverla de manera individual.**

**Nombre y matrícula:**

*   Enma Galindo
*   A01795095


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# Incluye aquí todas las librerías y paquetes que requieras.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold, cross_validate
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV, cross_validate, RepeatedStratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix



### **Liga de Datos de la UCI "Restaurant & consumer data".**
### **Descarga "RCdata.zip" y de ahí necesitaremos los archivos "rating_final.csv" y "geoplaces2.csv".**

https://archive.ics.uci.edu/dataset/232/restaurant+consumer+data



In [3]:
# Descarga los archivos de la página de la UCI a partir de los cuales generaremos
# nuestras matrices de utilidad.

data1 = pd.read_csv("/content/drive/MyDrive/MNA - Ciencia y Análisis de Datos /rating_final.csv", header='infer', sep=",")
data2 = pd.read_csv("/content/drive/MyDrive/MNA - Ciencia y Análisis de Datos /geoplaces2.csv", header='infer',  encoding='latin-1')

print(data1.shape, data2.shape)


(1161, 5) (130, 21)


## **Ejercicio - 1**

### **Explica cuál es el propósito de usar el argumento "latin-1" al cargar el segundo archivo.**

El propósito de utilizar el argumento "latin-1" al cargar el segundo archivo (geoplaces2.csv) es manejar adecuadamente cualquier carácter especial que pueda estar presente en el archivo.

Es fundamental especificar el tipo de codificación utilizada para escribir un archivo de texto. Al leer caracteres especiales, como letras acentuadas, caracteres que no son ASCII, etc., pueden producirse errores si no se especifican correctamente.

El estándar de codificación "latin-1" (también conocido como ISO-8859-1) es un conjunto de caracteres que incluye los caracteres ASCII estándar y muchos otros caracteres utilizados en los idiomas europeos. Es bastante fuerte y puede manejar una amplia gama de personajes especiales.


Por lo tanto, usar `encoding='latin-1'` asegura que los caracteres especiales presentes en el archivo `geoplaces2.csv` se lean correctamente, evitando así posibles errores de codificación al cargar el archivo en el DataFrame de pandas.

In [7]:
# Obtener las variables presentes en el DataFrame
variables = data1.columns

# Mostrar las variables
print(variables)

Index(['userID', 'placeID', 'rating', 'food_rating', 'service_rating'], dtype='object')


In [8]:
# Obtener las variables presentes en el DataFrame
variables = data2.columns

# Mostrar las variables
print(variables)

Index(['placeID', 'latitude', 'longitude', 'the_geom_meter', 'name', 'address',
       'city', 'state', 'country', 'fax', 'zip', 'alcohol', 'smoking_area',
       'dress_code', 'accessibility', 'price', 'url', 'Rambience', 'franchise',
       'area', 'other_services'],
      dtype='object')


In [4]:
# Del primer archivo obtenemos una matriz con 3 evaluaciones de los restaurantes:
# general, comida y servicio.
# Las evaluaciones pueden ser 0, 1 o 2, siendo 0 la menor calificación y 2 la
# mayor calificación:

data1.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [5]:
# Del segundo archivo obtenemos información diversa de cada restaurante:

data2.head(2)

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none


In [9]:
# De data1 no requeriremos "rating" y de data2 solo necesitamos "placeID" y "name":
# Definimos la lista y matrices con los factores que necesitamos:

lista_data1 = ['userID','placeID','food_rating','service_rating']
lista_data2 = ['placeID','name']

data1a = data1[lista_data1]
data2a = data2[lista_data2]

## **Ejercicio - 2**

In [10]:
# Define el DataFrame que conjunte la información de las dos DataFrame data1a
# y data2a en uno solo, a través de la columna común "placeID" y que
# llamaremos "df_combinado":



# ************* Inlcuye aquí tu código:*****************************


df_combinado = pd.merge(data1a, data2a, on='placeID')


# *********** Aquí termina la sección de agregar código *************


# Despleguemos la dimensión y los primeros renglones de este DataFrame:

print(df_combinado.shape)
df_combinado.head()

(1161, 5)


,userID,placeID,food_rating,service_rating,name
0,U1077,135085,2,2,Tortas Locas Hipocampo
1,U1108,135085,2,1,Tortas Locas Hipocampo
2,U1081,135085,2,1,Tortas Locas Hipocampo
3,U1056,135085,2,2,Tortas Locas Hipocampo
4,U1134,135085,1,2,Tortas Locas Hipocampo


## **Ejercicio - 3**

In [12]:
#    Define la matriz de utilidad cuyos renglones sean los nombres de los
#    restaurantes, las columnas los IDs de los usuarios y las entradas la
#    evaluación de la comida (food_rating). La llamaremos "UtMx_food".


# ************* Inlcuye aquí tu código:*****************************


UtMx_food = df_combinado.pivot_table(index='name', columns='userID', values='food_rating', fill_value=0)


# *********** Aquí termina la sección de agregar código *************


print('Dimensión de la matriz de Utilidad sobre la evaluación de la comida:')
print('(restaurantes, usuarios) =', (UtMx_food.shape))
UtMx_food.head()

Dimensión de la matriz de Utilidad sobre la evaluación de la comida:
(restaurantes, usuarios) = (129, 138)


userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
name,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Arrachela Grill,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cabana Huasteca,0,0,2,0,0,0,0,0,2,0,...,0,0,0,0,0,1,0,0,0,0
Cafe Chaires,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cafeteria cenidet,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## **Ejercicio - 4**

### **En la Factorización SVD la cantidad de valores singulares será menor o igual al menor valor de los renglones o columnas de la matriz de utilidad UtMx.**

### **La factorización SVD de una matriz $A$ tiene la forma:**

$A_{m\times n} = U_{m\times m}\Sigma_{m\times n}V_{n\times n}^T$

In [14]:
import pandas as pd
from sklearn.decomposition import TruncatedSVD

# Suponiendo que ya tienes la matriz de utilidad UtMx_food y el número máximo de valores singulares nc_food calculado

# Inicializamos y aplicamos la factorización SVD en relación a la evaluación de la comida
SVD_food = TruncatedSVD(n_components=nc_food)
SVD_food.fit(UtMx_food)

# Determinemos la cantidad de componentes que explican un 90% de la variabilidad acumulada de cada componente
for j in range(nc_food):
    if SVD_food.explained_variance_ratio_[0:j].sum() > 0.90:
        break

# Usaremos esta cantidad de componentes para las recomendaciones basadas en la calificación de la comida
N_food = j - 1

print('Total de valores singulares basados en evaluación de la comida:', nc_food)
print('Valor de truncamiento al 90% de dicha variabilidad:', N_food)


Total de valores singulares basados en evaluación de la comida: 129
Valor de truncamiento al 90% de dicha variabilidad: 51


In [15]:
# Apliquemos la transformación SVD para obtener la matriz de las variables
# latentes de los restaurantes en relación a la evalución de los usuarios con
# respecto a la comida. Deberás obtener la factorización SVD con el máximo de
# comonentes y por lo tanto el máximo de valores singulares posible.
# Asigna a la variable "nc_food" el valor del máximo número de valores singulares
# que podemos obtener de la utilidad que generamos en el ejercicio anterior.


# ************* Inlcuye aquí tu código:*****************************


nc_food = min(UtMx_food.shape)     # número de componentes


# *********** Aquí termina la sección de agregar código *************


# Inicializamos y aplicamos la factorización SVD en relación a la
# evaluación de la comida, para determinar la cantidad de componentes
# que podemos utilizar con un 90% de explicación de la variabilidad
# de dichas componentes:

SVD_food = TruncatedSVD( n_components = nc_food )
SVD_food.fit(UtMx_food)

# Determinemos la cantidad de componentes que explican un 90%
# de la variabilidad acumulada de cada componente:

for j in range(nc_food):
  if SVD_food.explained_variance_ratio_[0:j].sum() > 0.90:
    break

# Usaremos esta cantidad de componentes para las recomendaciones basadas
# en la calificación de la comida:
N_food = j-1

print('Total de valores singulares basados en evaluación de la :', nc_food)
print('Valor de truncamiento al 90% de dicha variabilidad:',N_food)


Total de valores singulares basados en evaluación de la : 129
Valor de truncamiento al 90% de dicha variabilidad: 51


In [16]:
# Usando la métrica de correlación de Pearson, obtendremos las 10 mejores
# recomendaciones de este modelo no supervisado con base a la información de
# alguien que desea obtener las similidades com el restaurante "Restaurante Pueblo
# Bonito" y con la cota de truncamiento obtenido en el ejercicio anterior.
# Para ello obtendremos las correlaciones positivas y de ahí deberás seleccionar
# las diez mejores.

# Factorización SVD:
SVD_food = TruncatedSVD(n_components = N_food)
resultant_matrix_food = SVD_food.fit_transform(UtMx_food)

# Matriz de correlación de Pearson:
corr_mat_food = np.corrcoef(resultant_matrix_food)

# Restaurante de referencia:
restaurante_de_referencia = "Restaurante Pueblo Bonito"
nombres_rest = UtMx_food.T.columns  # nombres de restaurantes
idx_rest = list(nombres_rest).index(restaurante_de_referencia)
corr_rest = corr_mat_food[idx_rest] # Vector de Correlación del RinconHuasteco

# Buscando las correlaciones positivas:
idx = (corr_rest>0)
mejores_sim_food = list()
for i in range(len(nombres_rest[idx])):
  mejores_sim_food.append((corr_rest[idx][i], nombres_rest[idx][i]))

print('Total de similaridades positivas encontradas:', len(mejores_sim_food))

print('Algunos de los resultados encontrados:')
mejores_sim_food[0:7]

Total de similaridades positivas encontradas: 72
Algunos de los resultados encontrados:


[(0.0046535859640976646, 'Arrachela Grill'),
 (0.023572145236466082, 'Cabana Huasteca'),
 (0.031032708168054868, 'Cafe Chaires'),
 (0.017132770777690425, 'Cafeteria cenidet'),
 (0.22425050020360382, 'Cafeteria y Restaurant El Pacifico'),
 (0.014007539558532298, 'Cenaduria El RincÃ³n de Tlaquepaque'),
 (0.004519714560598955, 'Chilis Cuernavaca')]

## **Ejercicio - 5**

In [17]:
# Ordenar la lista de recomendaciones "mejores_sim_food" encontrada en el paso
# anterior de manera descendente y llamaremos "mejores_sim_food_ordenadas".
#
# NOTA: Puedes incluir a continuación las líneas de código que requieras.

# ************* Inlcuye aquí tu código:*****************************



mejores_sim_food_ordenadas = sorted(mejores_sim_food, key=lambda x: x[1], reverse=True)



# *********** Aquí termina la sección de agregar código *************


# Desplegamos las 10 mejores similitudes encontradas de manera descendente:
print('Similitudes con base a la evaluación de la comida con mayores valores de correlación:')
for k in range(1,11):
  print('%d> %s' % (k, mejores_sim_food_ordenadas[k]))

Similitudes con base a la evaluación de la comida con mayores valores de correlación:
1> (0.021356823902355885, 'sirloin stockade')
2> (0.04386573786979212, 'shi ro ie')
3> (0.017090439227809978, 'rockabilly ')
4> (0.16358025928772502, 'puesto de tacos')
5> (0.700988057293103, 'pizza clasica')
6> (0.028588614053364356, 'palomo tec')
7> (0.26869332251368433, 'la Cochinita Pibil Restaurante Yucateco')
8> (0.029192086238757024, 'el pueblito')
9> (0.007693120363516159, 'el lechon potosino ')
10> (0.0016835183873567118, 'don burguers')


## **Ejercicio - 6**

### **De manera análoga y usando ahora la evaluación con respecto al servicio (service_rating), encuentrarás ahora los diez restaurantes con mayor similaridad al "Restaurante Pueblo Bonito" y con respecto a la métrica de similaridad de Pearson.**

### **Deberás usar la cantidad de componentes necesarias para que la varabilidad de la factorización SVD quede explicada en un 90%.**

### **a.  Para ello define primero la matriz de utilidad, UtMx_service, donde los renglones son los nombres de los restaurantes, las columnas los usuarios y las entradas las evaluaciones con respecto al servicio recibido en el restaurante.**





In [18]:
# a. Define la matriz de utilidad cuyos renglones sean los nombres de los
#    restaurantes, las columnas los IDs de los usuarios y las entradas la
#    evaluación del servicio (service_rating). La llamaremos "UtMx_service".


# ************* Inlcuye aquí tu código:*****************************


UtMx_service = df_combinado.pivot_table(index='name', columns='userID', values='service_rating', fill_value=0)


# *********** Aquí termina la sección de agregar código *************




print('Dimensión de la matriz de Utilidad sobre la evaluación del servicio:')
print('(restaurantes, usuarios) =', (UtMx_service.shape))
UtMx_service.head()

Dimensión de la matriz de Utilidad sobre la evaluación del servicio:
(restaurantes, usuarios) = (129, 138)


userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
name,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Arrachela Grill,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cabana Huasteca,0,0,2,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
Cafe Chaires,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cafeteria cenidet,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# b. Incluye el código necesario para obtener y desplegar los 10 restaurantes con
#    mayor similaridad de Pearson al "Restaurante Pueblo Bonito", y usando solo
#    el 90% de las componentes que explican la varianza de la matriz UtMx_service.
#    Desplegarlos en orden descendente, sin incliur el restaurante de referencia.

import numpy as np
from scipy.stats import pearsonr

# 1. Aplicar la factorización SVD a la matriz UtMx_service
SVD_service = TruncatedSVD(n_components=UtMx_service.shape[1] - 1)  # Utilizar todas las componentes menos una
SVD_service.fit(UtMx_service)

# Determinar el número de componentes necesarias para explicar el 90% de la varianza
explained_variance_ratio_cumsum = np.cumsum(SVD_service.explained_variance_ratio_)
n_components_90 = np.argmax(explained_variance_ratio_cumsum >= 0.90) + 1

# Reducir la dimensión de la matriz UtMx_service utilizando solo el 90% de las componentes
UtMx_service_reduced = SVD_service.transform(UtMx_service)[:, :n_components_90]

# 2. Calcular las similitudes de Pearson entre el "Restaurante Pueblo Bonito" y todos los demás restaurantes
restaurante_pueblo_bonito = 'Restaurante Pueblo Bonito'
pearson_similarities = {}
for restaurante in UtMx_service.index:
    if restaurante != restaurante_pueblo_bonito:
        similarity = pearsonr(UtMx_service.loc[restaurante_pueblo_bonito], UtMx_service.loc[restaurante])[0]
        pearson_similarities[restaurante] = similarity

# 3. Ordenar las similitudes de Pearson en orden descendente
pearson_similarities_sorted = sorted(pearson_similarities.items(), key=lambda x: x[1], reverse=True)

# 4. Mostrar los nombres de los 10 restaurantes con mayor similitud, excluyendo el "Restaurante Pueblo Bonito"
print('Los 10 restaurantes con mayor similitud de Pearson al "Restaurante Pueblo Bonito":')
for i, (restaurante, similitud) in enumerate(pearson_similarities_sorted[:10], start=1):
    print(f'{i}. {restaurante}: {similitud}')


Los 10 restaurantes con mayor similitud de Pearson al "Restaurante Pueblo Bonito":
1. pizza clasica: 0.5562209612350641
2. El Club: 0.4907891592188079
3. Restaurante la Estrella de Dima: 0.4111729271692411
4. Restaurante Guerra: 0.35876214604630385
5. Restaurant Orizatlan: 0.32147171424936255
6. la Cochinita Pibil Restaurante Yucateco: 0.2979096270258222
7. Restaurante El Cielo Potosino: 0.28900291411751206
8. Hamburguesas saul: 0.26938195741193066
9. Restaurant Oriental Express: 0.23012974345028817
10. La Fontana Pizza Restaurante and Cafe: 0.22869016836041064


## **Ejercicio - 7**

### **Incluye tus comentarios y conclusiones de la actividad. En particular indica cuáles restaurantes encontraste con correlaciones altas tanto para la calificación de la comida, como del servicio.**

1. Factorización de valores singulares (SVD) para la evaluación de la comida: Se utilizó la técnica de evaluación de valores singulares (SVD) para obtener la matriz de variables latentes de los restaurantes en relación con la evaluación de la comida por parte de los usuarios.
        - A partir de la matriz de número de restaurantes y usuarios de los servicios públicos se calculó el máximo número de valores singulares posibles.
        - Se calcula el número de componentes necesarios para explicar el 90% de la variabilidad de la matriz de utilidades.

2. Resultados de la factorización SVD para la evaluación de alimentos:
        - Los valores singulares totales basados en la evaluación dietética fueron 129, y se calculó que 51 componentes eran necesarios para explicar el 90% de la variabilidad.
        - La medida de evaluación de Pearson se utiliza para encontrar similitudes entre restaurantes.
        - Los restaurantes con correlaciones altas para la calificación culinaria incluyen "solomillo empalizada", "shi ro ie", "puesto de tacos" y "pizza clásica", entre otros.

3. Factorización SVD para evaluación del servicio:

        - Se definió la matriz de utilidad para evaluar el servicio, con los restaurantes como filas y los usuarios como columnas.
        - A esta matriz se le aplicó la factorización SVD y se redujo la dimensión para explicar la variación del 90%.

4. Resultados de la factorización SVD para la evaluación del servicio:
        - Se encontraron similitudes entre los restaurantes calculados en la evaluación del servicio utilizando la métrica de compensación de Pearson.
        - Entre los restaurantes con correlaciones altas para la calificación del servicio se encuentran “pizza clásica”, “El Club” y “Restaurante la Estrella de Dima”, entre otros.

Se utilizaron técnicas de factorización SVD y métricas de compensación de Pearson para encontrar similitudes entre restaurantes basándose en evaluaciones de comida y servicio. Esto permite identificar restaurantes similares para recomendaciones personalizadas y análisis de las preferencias de los usuarios. Los resultados proporcionan información útil para recomendaciones y análisis en el contexto de los sistemas de recomendación.

>> **Fin de la Actividad de Sistemas de Recomendación**